In [1]:
import os
import random
import numpy as np
import cv2
import json
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mtcnn.mtcnn import MTCNN
from tqdm.notebook import tqdm

# Settings

In [2]:
# set the size of the dataset, manually
samples = 10000
# set the target orientation
orientation = 1
# set the scale for resizing the images
scale = .5

# Helper Functions

In [143]:
# ----------------------------------------------------------------------
# function for reading the Json files
# ----------------------------------------------------------------------
def idx_finder(folder):
    '''
    This function takes the folder name (string),
    and returns a list of indices of the target
    orientation and valid frames in the folder.
    '''
    # set the specific folder's path
    path = 'Data_MIT/' + folder

    # read the json files
    # ------------------------------------------------------------------
    # read the Face Crop json
    with open(path + '/appleFace.json', 'r') as file:
        face = json.load(file)
    # read the Left Eye json
    with open(path + '/appleLeftEye.json', 'r') as file:
        eye_l = json.load(file)
    # read the Right Eye json
    with open(path + '/appleRightEye.json', 'r') as file:
        eye_r = json.load(file)
    # read the Scree json
    with open(path + '/screen.json', 'r') as file:
        screen = json.load(file)
    
    # make a list of all valid indices
    # ------------------------------------------------------------------
    # list of all frames with full face in it
    faces = [idx for idx, val in enumerate(face['IsValid']) if val == 1]
    # list of all frames with the left eye in it
    eyes_l = [idx for idx, val in enumerate(eye_l['IsValid']) if val == 1]
    # list of all frames with the right eye in it
    eyes_r = [idx for idx, val in enumerate(eye_r['IsValid']) if val == 1]
    # list of all portrait frames
    portraits = [idx for idx, ori in enumerate(screen['Orientation']) if ori == orientation]
    # intersection of all those lists
    indices = [idx for idx in faces if idx in eyes_l and idx in eyes_r and idx in portraits]
    
    return indices



# ----------------------------------------------------------------------
# a function for reading Json files
# ----------------------------------------------------------------------
def json_reader(dictionary, jsn, frame):
    '''
    The function takes a dictionary and a string mentioning
    the name of the json file, and a frame index, and it
    reads index-related info from json files and record them
    in the dictionary.
    '''
    # set the path to the json files
    path = 'Data_MIT/' + frame[:5] + '/'
    # get the index of the frame
    idx = int(frame[-5:])
    # read the face json file
    with open(str(path + 'apple' + jsn + '.json'), 'r') as file:
        jsn_info = json.load(file)    
    # add the face patch coordinates to the dictionary
    dictionary[jsn] = [round(jsn_info['X'][idx]),
                       round(jsn_info['Y'][idx]),
                       round(jsn_info['W'][idx]),
                       round(jsn_info['H'][idx])]


    
# ----------------------------------------------------------------------    
# a function to cut, resize, and save a patch
# ----------------------------------------------------------------------
def patch_reader(image, coordinates, path):
    '''
    The function takes an image and a list of coordinates (int),
    and a path (string), and cuts a patch out of the image,
    resizes it, and saves it in desired path.
    '''
    # check if the patches falls out of the frame
    # -------------------------------------------
    # whether the X coordinate is negative
    if coordinates[0] < 0:
        # if so, fill the shortage by replicating the edge 
        image = cv2.copyMakeBorder(image, 0, 0, np.abs(coordinates[0]), 0, cv2.BORDER_REPLICATE)
        # and reset the X value regarding the new frame
        coordinates[0] = 0

    # whether Y is negative
    if coordinates[1] < 0:
        # if so, fill the shortage by replicating the edge 
        image = cv2.copyMakeBorder(image, np.abs(coordinates[1]), 0, 0, 0, cv2.BORDER_REPLICATE)
        # and reset the Y value regarding the new frame
        coordinates[1] = 0

    # whether X+W is larger than the frame width
    if coordinates[0]+coordinates[2] > image.shape[1]:
        # if so, fill the shortage (diff) by replicating the edge
        diff = np.abs(image.shape[1]-(coordinates[0]+coordinates[2]))
        image = cv2.copyMakeBorder(image, 0, 0, 0, diff, cv2.BORDER_REPLICATE)

    # whether Y+H is larger than the frame height
    if coordinates[1]+coordinates[3] > image.shape[0]:
        # so we fill the shortage (diff) by replicating the edge
        diff = np.abs(image.shape[0]-(coordinates[1]+coordinates[3]))
        image = cv2.copyMakeBorder(image, 0, diff, 0, 0, 0, cv2.BORDER_REPLICATE)

        
    # cut the patch
    patch = image[coordinates[1]:coordinates[1]+coordinates[3],
                  coordinates[0]:coordinates[0]+coordinates[2]]
    
    # resize the patch
    patch_res = cv2.resize(patch, (224, 224))
    
    # save the image
    cv2.imwrite(path, patch_res);



# ----------------------------------------------------------------------
# a function for reading grid coordinates
# ----------------------------------------------------------------------
def get_coords(folder, frame):
    '''
    This function takes the folder and the frame names (string),
    read the faceGrid.json file, and returns the grid
    coordinates.
    '''
    # set the path to the grid json file
    path = 'Data_MIT/' + folder + '/faceGrid.json'
    # read the json file
    with open(path, 'r') as file:
        grid = json.load(file)
    # set the frame index
    idx = int(frame)
    # make a list of the X,Y,W, and H coordinates
    coords = [round(grid['X'][idx]), round(grid['Y'][idx]),
              round(grid['W'][idx]), round(grid['H'][idx])]
    
    # and return the coords list
    return coords



# ----------------------------------------------------------------------
# a function for creating the grids based on the paper
# ----------------------------------------------------------------------
def grid_paper(coords, name):
    '''
    This function takes a list of coordinates, and
    a name, and saves the face grid within a 25X25
    white frame. 
    '''
    # set the canvas
    canvas = np.ones((25,25))*255

    # grid coordinates
    x = coords[0]
    y = coords[1]
    w = coords[2]
    h = coords[3]
    
    # mask the face location
    canvas[y:y+h, x:x+w] = 0

    # set the path to save the grids
    path = 'Data_Mini/GridPaper/' + name + '.jpg'
    # save the face grid
    plt.imsave(path, canvas, cmap='gray');



# ----------------------------------------------------------------------
# define a function for making facial grid of an image using MTCNN
# ----------------------------------------------------------------------
def grid_mtcnn(path):
    '''
    The function takes a path (string) to a scaled image (320,240) of a face 
    using the MTCNN computes its facial landmarks, creates a white canvas
    of the same size as the original image and marks the facial landmarks
    with black face-box and white dots for eyes and nose, and saves the
    canvas (the gird) with the same name in the Grids folder. 
    '''
    # read the image
    img = plt.imread(path)
    # detect face keypoints using MTCNN
    results = MTCNN().detect_faces(img)
    # set a white canvas with the same dimensions as the image
    canvas = np.ones((320,240)) * 255
    # get the face bounding box parameters
    x, y, width, height = results[0]['box']
    # blacken all the pixels falling in the face bounding box
    canvas[y:y+height, x:x+width] = 0
    # get the keypoints for eyes and nose while ignoring the mouth two points
    keypoints = {key: val for key, val in results[0]['keypoints'].items() if not key.startswith('m')}
    # whiten the keypoints
    for key, val in keypoints.items():
        canvas[val[1],val[0]]=255
    # finally save it
    plt.imsave(p[:10] + 'GridMTCNN/' + p[-15:], canvas, cmap='gray')
    

# Create the Input

### Picking/Saving Random Frames

In [13]:
# make a list of the folders in the data directory
folders = [name for name in os.listdir('Data_MIT/')]
# drop the irrelevant file in the data directory
folders.remove('LICENSE.md')
folders.remove('README.md')

# create a list of valid and portrait frames in folders
frames_all = []
for folder in tqdm(folders):
    # make a list of valid frames
    indices = idx_finder(folder)
    frames = [folder + '-' + (5 - len(str(idx))) * '0' + str(idx) for idx in indices]
    frames_all = frames_all + frames

# pick the 'samples' number of the frames, randomly
# and in case of 'samples' larger than the number of
# all frames, pick them all
frames = random.sample(frames_all, k=min(samples, len(frames_all)))
frames.sort()

# create a dictionary of folders (keys) and frames (values-list)
# first create a list of the folders
folders = sorted(list(set([frame[:5] for frame in frames])))
# the create an empty dictionary for saving the frames data
frames_dic = {}
# iterate over the folders and store each folder's picked frames
for folder in folders:
    frames_dic[folder] = [frame[-5:] for frame in frames if frame[:5]==folder]
    
# save the dictionary
with open('Data_Mini/frames.pkl','wb') as f:
    pickle.dump(frames_dic, f)


### Extract/Save the Labels

In [72]:
# create a list to store dot location for each frame
labels = []


# iterate over dictionary keys/values and get the labels
for key, value in frames_dic.items():
    # set the specific folder's path
    path = 'Data_MIT/' + key
    
    # read the dot json
    with open(path + '/dotInfo.json', 'r') as file:
        dot = json.load(file)
    
    # convert the frames list from string to integer
    indices_int = [int(idx) for idx in value] 
    
    for idx in indices_int:
        labels.append([dot['XCam'][idx], dot['YCam'][idx]])

        
# make an np.array out of the coordinates for valid frames
labels = np.array(labels)

# and save it in a pickle file
with open('Data_Mini/labels.pkl','wb') as f:
    pickle.dump(labels, f)

### Extract/Save the Resized Images

In [77]:
# iterate over folders in dict and frames in folders
for key, value in tqdm(frames_dic.items()):
    for idx in value:
        # set the path to each frame
        path = 'Data_MIT/' + key + '/frames/' + idx + '.jpg'
        
        # read the image
        img = cv2.imread(path)
        
        # resize the image
        new_h = int(img.shape[1]*scale)
        new_w = int(img.shape[0]*scale)
        img = cv2.resize(img, (new_h, new_w))
    
        # set the address for saving the image
        save_path = 'Data_Mini/Images/' + key + '-' + idx + '.jpg'  
        
        # and save the image
        cv2.imwrite(save_path, img);


### Extract/Save the Patches

In [101]:
# list of the json files to read
json_list = ['Face', 'LeftEye', 'RightEye']


# iterate over folders in dict and frames in folders
for key, value in tqdm(frames_dic.items()):
    for idx in value:
        
        # define the dictionary for recording the info
        coords = {}
        
        # iterate over all json files and read/record the info
        for jsn in json_list:
            # read each json file
            json_reader(coords, jsn, key + '-' + idx)

        # adjust the eyes' coordinates to express the location
        # for within the frame
        coords['LeftEye'][0] += coords['Face'][0]
        coords['LeftEye'][1] += coords['Face'][1]
        coords['RightEye'][0] += coords['Face'][0]
        coords['RightEye'][1] += coords['Face'][1]

        # set the path to the image
        path = 'Data_MIT/' + key + '/frames/' + idx + '.jpg'
        # read the image
        img = cv2.imread(path)
        
        # iterate over two eyes
        for patch in ['Face', 'LeftEye', 'RightEye']:
            # set the save path
            save_path = 'Data_Mini/' + patch + '/' + key + '-' + idx + '.jpg' 
            # save the patch
            patch_reader(img, coords[patch], save_path)


### Extract/Save the Paper Grids

In [138]:
# iterate over folders in dict and frames in folders
for key, value in tqdm(frames_dic.items()):
    for idx in value:
        # get the grid coordinates
        coords = get_coords(key, idx)
        # save the grid
        grid_paper(coords, key + '-' + idx )


### Extract/Save the MTCNN Grids

In [ ]:
# iterate over folders in dict and frames in folders
for key, value in frames_dic.items():
    for idx in value:
        # set the path to the image
        path = 'Data_Mini' + key + '-' + idx + '.jpg'
        # create and save the grid
        grid_mtcnn(path)
        

# Save Data-Pickle Files